In [4]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer, InputExample, losses, models
from torch.utils.data import DataLoader
import torch

/Users/likhit/Documents/Coding/NLP/NLP_basics/nlp/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm



🧱 Transformer Layers:
Layer 0
Layer 1
Layer 2
Layer 3
Layer 4
Layer 5
✅ Using device: mps


 33%|███▎      | 500/1500 [12:33<28:19,  1.70s/it]                   

{'loss': 0.1641, 'grad_norm': 1.2210733890533447, 'learning_rate': 1.3378378378378381e-05, 'epoch': 3.33}


 67%|██████▋   | 1000/1500 [23:53<10:44,  1.29s/it]

{'loss': 0.0909, 'grad_norm': 1.3204602003097534, 'learning_rate': 6.621621621621622e-06, 'epoch': 6.67}


100%|██████████| 1500/1500 [34:58<00:00,  1.40s/it]


{'loss': 0.0714, 'grad_norm': 1.5489064455032349, 'learning_rate': 0.0, 'epoch': 10.0}
{'train_runtime': 2098.7825, 'train_samples_per_second': 45.731, 'train_steps_per_second': 0.715, 'train_loss': 0.1088121821085612, 'epoch': 10.0}

✅ Training complete. Model saved to 'fineeeeminilm_proj512_only_dense'


In [ ]:
dataset = load_dataset("neural-bridge/rag-dataset-12000", split="train")
pairs = [(q, c) for q, c in zip(dataset["question"], dataset["context"]) if isinstance(q, str) and q.strip() and isinstance(c, str) and c.strip()]
train_samples = [InputExample(texts=[q, c]) for q, c in pairs]

In [ ]:
word_embedding_model = models.Transformer("sentence-transformers/all-MiniLM-L6-v2")
hf_model = word_embedding_model.auto_model


print("Transformer Layers:")
layer_names = set()
for name, _ in hf_model.named_parameters():
    if "encoder.layer." in name:
        layer_num = name.split("encoder.layer.")[1].split(".")[0]
        layer_names.add(int(layer_num))

for param in hf_model.parameters():
    param.requires_grad = False

# Unfreezing last 2 layers
last_two = sorted(layer_names)[-2:]
for name, param in hf_model.named_parameters():
    if any(f"layer.{i}" in name for i in last_two):
        param.requires_grad = True


In [ ]:
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
print(f"✅ Using device: {device}")

train_dataloader = DataLoader(train_samples, shuffle=True, batch_size=64)
train_loss = losses.MultipleNegativesRankingLoss(model).to(device)
model.to(device)

In [ ]:
model.fit(
    train_objectives=[(train_dataloader, train_loss)],
    epochs=10,
    warmup_steps=10,
    show_progress_bar=True
)

model.save("partial_layer_frezzing")
print("Training complete. Model saved")
